# Segmenting and Clustering Neighborhoods in Toronto

In [ ]:
import pandas as pd 
import numpy as np
!pip install beautifulsoup4
!pip install lxml
import requests 
import random 

In [ ]:
from geopy.geocoders import Nominatim
from IPython.display import Image 
from IPython.core.display import HTML 
from IPython.display import display_html
import pandas as pd
import numpy as np
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes

In [ ]:
import folium # plotting library
from bs4 import BeautifulSoup

print('Folium installed')
print('Libraries imported.')

In [ ]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
bs=BeautifulSoup(source,'lxml')

In [ ]:
import matplotlib.cm as cm
import matplotlib.colors as colors
print ('Matplotlib Libraries imported')

In [ ]:
from sklearn.cluster import KMeans

In [ ]:
from IPython.display import display_html
tab = str(bs.table)
display_html(tab,raw=True)

In [ ]:
# Convert table to pandas dataframe
dfs = pd.read_html(tab)
df=dfs[0]

In [ ]:
# Cleaning Data
df_d = df[df.Borough != 'Not assigned']

df_c = df_d.groupby(['Postal Code','Borough'], sort=False).agg(', '.join)
df_c.reset_index(inplace=True)

df_c['Neighborhood'] = np.where(df_c['Neighborhood'] == 'Not assigned',df_c['Borough'], df_c['Neighborhood'])

df_c.head(12)

In [ ]:
df_c.shape

In [ ]:
# Importing the csv file conatining the latitudes and longitudes
lat_lon = pd.read_csv('https://cocl.us/Geospatial_data')
lat_lon.head()

In [ ]:
# Merging the tables to the Latitudes and Longitudes table
df_m = pd.merge(df_c,lat_lon,on='Postal Code')
df_m.head()

In [ ]:
# The rows with key word Toronto.
df_t = df_m[df_m['Borough'].str.contains('Toronto',regex=False)]
df_t

In [ ]:
#Visualizing all the Neighbourhoods of the above data frame using Folium
Toronto_m = folium.Map(location=[43.651070,-79.347015],zoom_start=10)

for lat,lng,Borough,Neighborhood in zip(df_t['Latitude'],df_t['Longitude'],df_t['Borough'],df_t['Neighborhood']):
    label = '{}, {}'.format(Neighborhood, Borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
    [lat,lng],
    radius=5,
    popup=label,
    color='blue',
    fill=True,
    fill_color='#3186cc',
    fill_opacity=0.7,
    parse_html=False).add_to(Toronto_m)
Toronto_m

In [ ]:
#KMeans clustering for the neighbourhoods
k=5
clustert = df_t.drop(['Postal Code','Borough','Neighborhood'],1)
kmeans = KMeans(n_clusters = k,random_state=0).fit(clustert)
kmeans.labels_

In [ ]:
df_t.drop(["C Labels"], axis = 1, inplace = True) 
df_t

In [ ]:
# create map
clusm = folium.Map(location=[43.651070,-79.347015],zoom_start=10)

# set color scheme for the clusters
x = np.arange(k)
ys = [i + x + (i*x)**2 for i in range(k)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# Markers
markers_colors = []
for lat, lon, Neighborhood, cluster in zip(df_t['Latitude'], df_t['Longitude'], df_t['Neighborhood'], df_t['Cluster Labels']):
    label = folium.Popup(' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(clusm)
       
clusm